# Pivot Tables in Pandas

## Learning Objectives

<div style="border: 1px solid black; padding: 10px; background-color: #EDEDED;">
    <ul>
        <li>Explain the concept of unique values and why it is so important.</li>
        <li>Create a simple pivot table in Pandas that computes one statistic for the unique values of one column.</li>
        <li>Create a pivot table in Pandas that computes multiple statistics for the unique values of one column.</li>
        <li>Create a pivot table in Pandas that groups by multiple variables within rows.</li>
        <li>Create a pivot table in Pandas that groups by two variables organized into rows and columns.</li>
        <li>Create a pivot table in Pandas that computes multiple statistics for the unique values of one column.</li>
        <li>Explain how to create more complicated pivot tables by using the <i>index</i>, <i>columns</i>, <i>values</i>, and <i>aggfunc</i> arguments.</li>
        <li>Use the <i>len</i> function to count the number of rows that exist for unique values in different columns.</li>
        <li>Explain how a Pandas pivot table is a DataFrame, and can be manipulated just like any other DataFrame.</li>
    </ul>
</div>

## Overview of Pivot Tables in Pandas

In previous units, we showed you how to _prepare_ your data for analysis. You loaded, cleaned, filtered, and manipulated data. In the previous chapter, we showed you how to begin data analysis by computing descriptive statistics. In this chapter, we will continue our journey into analyzing data by teaching you about pivot tables in Pandas.

A pivot table is a table that provides statistics for unique values of one or more variables. For example, what is the average salary in each department of a company? You will find that pivot tables will be one of the most useful tools in your toolkit. If you are adept at making pivot tables, you will be a star in your job. However, as you learn about pivot tables, realize that the skill lies not in writing code, but in thinking about what you want to learn from your data and interpreting the results. If you can envision the pivot table needed for your business question before you write any code, then the act of creating the table will seem simple.

## Imports and Loading the Data

In [1]:
import numpy as np
import pandas as pd

In this chapter, we will work with salary data for employees of the City of New York ([link](https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e)).

The cell below loads salary data for employees of the City of New York. Each row represents one employee in the one fiscal year. For a list of the columns and their meanings, click [here](https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e).

<b>NOTE: </b>You need either the <i>fastparquet</i> library or the <i>pyarrow</i> libraryto load the data file used in these notes.

You should have installed one of these libraries at the beginning of the semester when you installed Python. If you did not, or if you get errors, don't panic. <i>If you just want to read these notes, you do not need to run these cells and you do not need to install either library.</i>

If you're wondering what these libraries do, read this paragraph. The data file is quite large, approximately 376 MB as a CSV. That makes it difficult to share. It has too many rows to store in an Excel worksheet, and if we gave you a CSV, it would take many minutes to load, and you would have to clean many of the columns by converting them from text to other data types. To avoid all this, we chose to store the data in [Apache Parquet](http://parquet.apache.org/) format. Parquet is a super-cool format that stores the data by column. That allows it to easily compress the data, because you tend to find the same value repeated many times in each column. This compression reduces the file size (the parquet file is just under 50 MB), and greatly speeds up the loading process. We love the Parquet format and frequently use it when working with larger datasets. It is compatible with many big data and cloud technologies. And Pandas provides support for Parquet.

In [2]:
dfPay = pd.read_parquet('data/NYC_Payroll_data.parquet')
dfPay.head()

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,2016,NaN,DISTRICT ATTORNEY-MANHATTAN,ABA'AHMID,RAHASHEEM,E,2003-07-14 00:00:00+00:00,MANHATTAN,COMMUNITY ASSOCIATE,ACTIVE,47678.0,PER ANNUM,1830.00,47166.03,2.25,57.28,0.0
1,2016,NaN,DISTRICT ATTORNEY-MANHATTAN,ABENSUR,MARGARET,,1995-06-12 00:00:00+00:00,MANHATTAN,ADMINISTRATIVE ACCOUNTANT,ACTIVE,119959.0,PER ANNUM,1831.00,119042.50,0.00,0.00,0.0
2,2016,NaN,DISTRICT ATTORNEY-MANHATTAN,ABOUNAOUM,ANDREA,L,2011-10-11 00:00:00+00:00,MANHATTAN,COMMUNITY ASSOCIATE,ON LEAVE,39966.0,PER ANNUM,1181.68,27452.62,1.00,21.34,33.0
3,2016,NaN,DISTRICT ATTORNEY-MANHATTAN,ABRAHAM,JONATHAN,J,2014-12-01 00:00:00+00:00,MANHATTAN,COMPUTER SYSTEMS MANAGER,ACTIVE,116000.0,PER ANNUM,1831.00,115104.57,0.00,0.00,0.0
4,2016,NaN,DISTRICT ATTORNEY-MANHATTAN,ABRAMS,JOSEPH,,2015-05-21 00:00:00+00:00,MANHATTAN,COLLEGE AIDE,CEASED,1.0,PER HOUR,0.00,3500.00,0.00,0.00,0.0


In [3]:
dfPay.shape

(2748818, 17)

## Unique / Distinct Values

Before we can teach you about pivot tables, we need to teach you an important underlying concept, unique values.

Consider the following randomly chosen values from the _Agency Name_ column of our dataset:

In [4]:
dfPay[['Work Location Borough']].sample(5, random_state=1)

,Work Location Borough
1686946,BRONX
535165,MANHATTAN
3128672,MANHATTAN
1632295,QUEENS
3092208,MANHATTAN


Notice that certain values are repeated. For example, "MANHATTAN" appears 3 times. What if we want to know all values that appear at least once in the column? We don't want to see repeats. We simply want to know which values appear at least once in the column. Why would we care to know this? First of all, because we might want a list of the boroughs in New York City! Second, when we create a pivot table, we might want to report statistics by borough. That means that each borough should appear only once in our pivot table.

What we are looking for are the _unique_ values in the column (note that the term _distinct_, which is used in SQL, is synonymous with _unique_). Pandas makes it very easy to compute the unique values in a column. Simply call the `unique` method:

In [5]:
dfPay['Work Location Borough'].unique()

array(['MANHATTAN', 'STATEN ISLAND', 'QUEENS', 'BRONX', 'BROOKLYN'],
      dtype=object)

Another way to get unique values in a column is the `value_counts` method. That method returns the unique values as well as a count of each value.

In [6]:
dfPay['Work Location Borough'].value_counts()

Work Location Borough
MANHATTAN        1975186
QUEENS            314960
BROOKLYN          270173
BRONX             150150
STATEN ISLAND      38349
Name: count, dtype: int64

We prefer the `value_counts` method to the `unique` method for two reasons. First, `value_counts` returns counts in addition to unique values and these are often useful. For example, we just learned that most of the employees in the dataset work in Manhattan. Second, `value_counts` returns a Pandas Series while `unique` returns a NumPy array. Since we are doing most of our work in Pandas, it's often easier to work directly with Pandas objects.

**The concept of unique values is super important.** Not only does it underlie the pivot tables that we are about to create, but it is also used when merging data tables. 

Let's see one more example of uniqueness. Let's retrieve the unique departments in the NYC government. From this list, which is quite long (there appear to be 160 departments in the NYC government!), we see that education is largest, followed by police.

In [7]:
dfPay['Agency Name'].value_counts()

Agency Name
DEPT OF ED PEDAGOGICAL            471072
DEPT OF ED PER SESSION TEACHER    426514
POLICE DEPARTMENT                 307717
DEPT OF ED PARA PROFESSIONALS     176199
BOARD OF ELECTION POLL WORKERS    170968
                                   ...  
BROOKLYN COMMUNITY BOARD #14          15
BRONX COMMUNITY BOARD #12             15
BRONX COMMUNITY BOARD #5              14
BRONX COMMUNITY BOARD #1              11
BRONX COMMUNITY BOARD #3              10
Name: count, Length: 160, dtype: int64

## Pivot Tables in Pandas

A pivot table computes something for unique values. That's all there is to it! :)

### A Simple Pivot Table

Let's begin with a simple example: let's compute the average salary paid to workers in each of NYC's boroughs. We'll first filter it to include only fiscal year 2018:

In [8]:
# Let's get cute and display the numbers in accounting format. :)
pd.set_option('float_format', lambda x: f'${x:,.0f}')

pd.pivot_table(data=dfPay[dfPay['Fiscal Year']==2018],
               index='Work Location Borough',
               values='Base Salary',
               aggfunc='mean')

,Base Salary
Work Location Borough,
BRONX,"$49,584"
BROOKLYN,"$52,777"
MANHATTAN,"$40,780"
QUEENS,"$52,138"
STATEN ISLAND,"$53,919"


##### What the pivot table shows and how it computes its values

Before we examine the code, let's look at the pivot table. There's one row per borough. The pivot table computed the unique values in the _Work Location Borough_ column and created one row per unique borough in the resulting pivot table. Then, for each borough, it filtered the data for that borough and computed the average of the base salary for that borough. To see this, consider the following code that produces the exact same values as our pivot table above.

In [9]:
# Get only the 2018 rows. Drop any rows with missing values
df2018 = dfPay[dfPay['Fiscal Year'] == 2018].dropna(subset=['Base Salary'])
# Now get the unique boroughs as a list
boroughs = df2018['Work Location Borough'].unique().tolist()
# Sort the list of boroughs
boroughs.sort()

# For each borough in the list of unique boroughs
for borough in boroughs:
    # Filter the 2018 data on that borough
    df = df2018[df2018['Work Location Borough'] == borough]
    # Compute the mean salary for that borough in 2018
    meanSalary = df['Base Salary'].mean()
    
    print(f"{borough}: ${meanSalary:,.0f}")

BRONX: $49,584
BROOKLYN: $52,777
MANHATTAN: $40,780
QUEENS: $52,138
STATEN ISLAND: $53,919


To reiterate, the pivot table computed the unique values in the _Work Location Borough_ column. For each unique borough, it filtered the data for that borough and then computed the average of the _Base Salary_ for that borough.

##### Understanding the code to generate the pivot table

Above, we used the following code to create our pivot table:
```python
pd.pivot_table(data=dfPay[dfPay['Fiscal Year']==2018],
               index='Work Location Borough',
               values='Base Salary',
               aggfunc='mean')
```
Let's examine the code and understand what each argument does.
* `pd.pivot_table`. This is the Pandas function for creating a pivot table.
* `data=dfPay[dfPay['Fiscal Year']==2018]`. You must pass a DataFrame to the `data` argument. This argument is required. The `pivot_table` function will use this data to create a pivot table.
* `index='Work Location Borough'`. The `index` argument tells the pivot table which values to use as rows in the resulting pivot table. In this case, we told it to take the _Work Location Borough_ column of our source data and extract its unique values.
* `values='Base Salary`. The pivot table will take the values from this column and use them in its computations. 
* `aggfunc='mean'`. "aggfunc" stands for "aggregation function". By passing the string `'mean'`, we told Pandas to use the Numpy function _numpy.mean_ as our aggregation function. Thus, the pivot table will average the values in the _Base Salary_ column.

In sum, the code tells Pandas to take the 2018 data, extract the unique boroughs and, for each borough, compute the mean of the _Base Salary_ column.

### Computing More Than One Statistic

In the above example, we compute the mean salary paid to workers in each borough in 2018. We will extend that example and show how to compute the mean _and_ median salary for each borough. Look at the following code and its output.

In [10]:
pd.pivot_table(data=dfPay[dfPay['Fiscal Year']==2018],
               index='Work Location Borough',
               values='Base Salary',
               aggfunc=['mean', 'median'])

,mean,median
,Base Salary,Base Salary
Work Location Borough,,
BRONX,"$49,584","$48,666"
BROOKLYN,"$52,777","$50,604"
MANHATTAN,"$40,780","$35,977"
QUEENS,"$52,138","$50,604"
STATEN ISLAND,"$53,919","$55,024"


Notice that the first data column, _mean Base Salary_, contains the exact same information as our table above. However, this new table contains a second column that contains the median salaries. 

The only change to the code was to the _aggfunc_ argument. We changed that to `aggfunc=['mean', 'median']`. Instead of passing a function, _we passed a list of functions_. The _pivot_table_ function then created one column per function. It did everything else identically. It first retrieved the unique boroughs (from the _index_ argument), used values from the _Base Salary_ column in its computations. For each borough, it computed the mean and the median.

### Grouping By Multiple Variables in Rows

Often, your business question implies a logical hierarchy among your variables. For example, say you want to see the average pay in each department, but you want to break it out by borough. Department heads might want to see this information to know if they are paying appropriate amounts for employees in different parts of the city. Another example is to show the pay by borough and break it out by hourly and salaried employees. This might be useful if one group of employees is unionized and the other is not. Or if the hourly and salaried employees belong to different unions, or if they have different contracts.

In this section, let's use the example of pay by department and by borough. To make this tractable, we will restrict our analysis to (a) salaried employees (b) in the top 5 departments (c) in 2018.

First, let's filter the data so we only have rows for salaried employees in 2018 for the top 5 departments. We'll save the filtered data in a data frame called _dfTop5Dept2018_. 

In [11]:
# Condition 1: fiscal year == 2018
# Condition 2: pay basis == 'PER ANNUM'
# Condition 3: get the top 5 departments, as measured by number of employees.
#     Save the top 5 departments in the variable top5depts
top5depts =  dfPay[(dfPay['Fiscal Year']==2018) & 
                   (dfPay['Pay Basis'] == 'PER ANNUM')]['Agency Name'].value_counts().head(5).index

# Filter the data frame dfPay using the 3 conditions above.
dfTop5Dept2018 = dfPay[(dfPay['Fiscal Year']==2018) & 
                       (dfPay['Pay Basis'] == 'PER ANNUM') & 
                       (dfPay['Agency Name'].isin(top5depts))]

Now let's create a pivot table showing, for each of the top 5 departments in 2018, the average pay in each borough.

In [12]:
pd.pivot_table(data=dfTop5Dept2018, 
               index=['Agency Name', 'Work Location Borough'], 
               values='Base Salary', 
               aggfunc='mean')

Base Salary
Agency Name                   Work Location Borough             
DEPT OF ED PARA PROFESSIONALS MANHATTAN                  $36,139
DEPT OF ED PEDAGOGICAL        MANHATTAN                  $87,796
FIRE DEPARTMENT               BRONX                      $72,784
                              BROOKLYN                   $77,556
                              MANHATTAN                  $76,143
                              QUEENS                     $71,143
                              STATEN ISLAND              $80,794
HRA/DEPT OF SOCIAL SERVICES   BRONX                      $47,447
                              BROOKLYN                   $54,658
                              MANHATTAN                  $61,390
                              QUEENS                     $46,781
                              STATEN ISLAND              $49,353
POLICE DEPARTMENT             BRONX                      $71,268
                              BROOKLYN                   $68,800
                              MANHATTAN                  $78,636
                              QUEENS                     $68,304
                              STATEN ISLAND              $73,843

Take a look at the above pivot table. Notice that the _Agency Name_ and _Work Location Borough_ columns are organized into a hierarchy. For each unique level of _Agency Name_, the pivot table shows all unique values of _Work Location Borough_. Notice that Pandas omitted boroughs with no employees.

From this table, we can see that two of the top departments in the city, Department of Ed Para Professionals and Dept of Ed Pedagogical, have all of their employees in Manhattan. For the other three departments, we can easily compare average salaries across boroughs. However, we cannot conclude from this that employees in some boroughs get paid more _for the same job_ than their counterparts in other boroughs. To examine that, we would need to compare job titles and years of experience.

##### Understanding the Unique Values when Grouping By Multiple Variables

When you group by multiple variables, you need to think of unique values differently. In the pivot table above, the unique values are (DEPT OF ED PARA PROFESSIONALS, MANHATTAN), (DEPT OF ED PEDAGOGICAL, MANHATTAN), (FIRE DEPARTMENT, BRONX), (FIRE DEPARTMENT, BROOKLYN), etc. In other words, take the unique values from _Agency Name_ and the unique values from _Work Location Borough_; form every possible combination of unique values. The pivot table computes the mean salary for each unique combination.

##### Understanding the Code to Create a Pivot Table that Groups By Multiple Variables

The code to create the pivot table above was:
```python
pd.pivot_table(data=dfTop5Dept2018, 
               index=['Agency Name', 'Work Location Borough'], 
               values='Base Salary', 
               aggfunc='mean')
```

If you compare this code to the code from our simplest example, you will discover only one difference. For the _index_ argument, we passed a list of column names. That tells Pandas to group the data first by _Agency Name_ and then by _Work Location Borough_.

### Grouping By Multiple Variables in Rows and Columns

Often, you want to compute something for two variables but there is no logical hierarchy among the variables. For example, say you want to show the average pay by borough and by fiscal year. We could use a format like that in the previous section, but should we group first by borough and then fiscal year? Or by fiscal year and then borough? There's no obvious answer. Also, the format shown above does not facilitate comparisons across the second variable. When you find yourself in a situation like this, consider organizing the data into rows and columns.

Take a look at the pivot table below:

In [13]:
pd.pivot_table(data=dfPay, 
               index='Work Location Borough', 
               columns='Fiscal Year', 
               values='Base Salary', 
               aggfunc='mean', 
               margins=True)

Fiscal Year,2014,2015,2016,2017,2018,2019,All
Work Location Borough,,,,,,,
BRONX,"$58,788","$44,626","$46,477","$44,402","$49,584","$51,025","$47,251"
BROOKLYN,NaN,"$48,105","$50,254","$49,710","$52,777","$53,834","$50,901"
MANHATTAN,"$62,937","$32,795","$37,827","$38,471","$40,780","$39,591","$37,889"
QUEENS,"$71,759","$47,722","$50,650","$48,483","$52,138","$52,358","$50,339"
STATEN ISLAND,"$69,804","$49,004","$51,869","$53,531","$53,919","$54,322","$52,599"
All,"$63,752","$36,720","$41,248","$41,382","$43,979","$43,116","$41,311"


The above table lays out the data by borough (rows) and fiscal year (columns). Notice that we included an average for each row (the _All_ column), and an average for each column (the _All_ row). This format allows for easy comparison across boroughs, across fiscal years, and within boroughs. It is a very efficient way to present data.

##### Understanding the Unique Values When Grouping By Multiple Variables in Rows and Columns

The unique values for the pivot table are all combinations of unique values of the rows and columns. Thus, (BRONX, 2014) is a unique value, as is (STATEN ISLAND, 2019). The pivot table will compute the mean of the base salary for each of these unique values.

##### Understanding the Code to Create a Pivot Table that Groups By Multiple Variables in Rows and Columns

Let's examine the code used to create the pivot table:
```python
pd.pivot_table(data=dfPay, 
               index='Work Location Borough', 
               columns='Fiscal Year', 
               values='Base Salary', 
               aggfunc='mean', 
               margins=True)
```

When you compare this to the simplest pivot table above, you will discover two differences:
1. We added a _columns_ argument. This tells Pandas to get the unique values from the _Fiscal Year_ column and use them as column headings in the pivot table.
2. We added the argument `margins=True`. That tells Pandas to include summary rows and columns in the pivot table (the ones with the heading _All_).

### Computing Statistics for Multiple Variables

Another use case for pivot tables is to compute the same statistics for multiple variables. As a simple example, let's compute the mean salary and mean years of experience by borough.

First, we need to compute the years of experience. Let's use New York's 2018 fiscal year, which ran from July 1, 2017 through June 30, 2018. We will compute the number of years of experience at the end of the fiscal year. That way, people who start in the middle of the fiscal year will show up as having a positive amount of work experience. We will also focus on salaried employees only.

In [14]:
# Filter on fiscal year 2018, salaried employees only
dfFilt = dfPay[(dfPay['Fiscal Year'] == 2018) & 
               (dfPay['Pay Basis'] == 'PER ANNUM')].copy()
# Create a date constant containing the end of the fiscal year
endOfFY = pd.to_datetime('6-30-2018', format='%m-%d-%Y', utc=True)

dfFilt['Years of Experience'] = ((endOfFY - dfFilt['Agency Start Date']) / np.timedelta64(1,'D')) / 365.0

In [15]:
# Reset the Pandas display options
pd.reset_option('float_format')

pd.pivot_table(data=dfFilt,
               index='Work Location Borough',
               values=['Base Salary', 'Years of Experience'],
               aggfunc='mean')

,Base Salary,Years of Experience
Work Location Borough,,
BRONX,65145.129919,11.126592
BROOKLYN,67400.887592,11.648366
MANHATTAN,75038.574755,12.030538
QUEENS,68833.177272,10.573341
STATEN ISLAND,70285.517079,12.282185


In the above code, we passed a list of column names to the _Values_ argument. That told Pandas to compute the mean of each _Values_ column for each unique value in _Work Location Borough_.

## More Complicated Pivot Tables

In this chapter, we showed you how to create simple pivot tables in Pandas. You can create much more complicated tables by passing lists to multiple arguments in the `pivot_table` function. For example, say we want to recreate a pivot table that breaks out Pay Basis by Borough (rows). In each column, we want to show a different fiscal year. And we want to do this for two variables, _Base Salary_ and _Total Other Pay_. You could do that as follows. Notice that all we're doing is passing more columns to the arguments than we did previously.

In [16]:
pd.pivot_table(data=dfPay,
               index=['Work Location Borough', 'Pay Basis'],
               columns='Fiscal Year',
               values=['Base Salary', 'Total Other Pay'],
               aggfunc='mean')

Base Salary                \
Fiscal Year                                    2014          2015   
Work Location Borough Pay Basis                                     
BRONX                 PER ANNUM        58914.094319  60474.766639   
                      PER DAY                   NaN    192.062425   
                      PER HOUR            29.265000     22.159489   
                      PRORATED ANNUAL           NaN  26774.639752   
BROOKLYN              PER ANNUM                 NaN  61959.211599   
                      PER DAY                   NaN    228.706633   
                      PER HOUR                  NaN     19.998751   
                      PRORATED ANNUAL           NaN  27528.794118   
MANHATTAN             PER ANNUM        70302.205923  66926.799450   
                      PER DAY            278.545333     60.920634   
                      PER HOUR             3.790839      8.753896   
                      PRORATED ANNUAL           NaN  30113.815237   
QUEENS                PER ANNUM        72638.769939  64872.518101   
                      PER DAY                   NaN    254.609522   
                      PER HOUR            28.836250     26.367431   
                      PRORATED ANNUAL           NaN  29193.782209   
STATEN ISLAND         PER ANNUM        73064.887850  65302.354034   
                      PER DAY                   NaN    306.503156   
                      PER HOUR            31.514000     18.891888   
                      PRORATED ANNUAL           NaN  26722.350649   

                                                                   \
Fiscal Year                                    2016          2017   
Work Location Borough Pay Basis                                     
BRONX                 PER ANNUM        62271.607774  64138.005775   
                      PER DAY            209.050852    174.560338   
                      PER HOUR            23.570725     27.410545   
                      PRORATED ANNUAL  24266.495614  26549.752066   
BROOKLYN              PER ANNUM        63611.989467  66203.370088   
                      PER DAY            250.435743    230.142034   
                      PER HOUR            21.914079     23.865114   
                      PRORATED ANNUAL  25489.415011  32011.370213   
MANHATTAN             PER ANNUM        69219.929993  72014.764937   
                      PER DAY             66.200247     66.778423   
                      PER HOUR             9.441464   1167.428688   
                      PRORATED ANNUAL  32451.825311  33278.138812   
QUEENS                PER ANNUM        66468.231356  67518.476146   
                      PER DAY            274.408341    261.250382   
                      PER HOUR            30.445100     32.692165   
                      PRORATED ANNUAL  29777.313043  29397.394958   
STATEN ISLAND         PER ANNUM        67092.591946  69204.875021   
                      PER DAY            326.049903    346.949620   
                      PER HOUR            21.512435     22.148017   
                      PRORATED ANNUAL  23334.315789  29979.666667   

                                                                   \
Fiscal Year                                    2018          2019   
Work Location Borough Pay Basis                                     
BRONX                 PER ANNUM        65145.129919  65869.988143   
                      PER DAY            242.845655    224.684363   
                      PER HOUR            24.212095     23.177066   
                      PRORATED ANNUAL  27333.981685  26009.958333   
BROOKLYN              PER ANNUM        67400.887592  68356.687433   
                      PER DAY            273.271984    277.464870   
                      PER HOUR            22.591124     23.095998   
                      PRORATED ANNUAL  28857.308000  29393.553125   
MANHATTAN             PER ANNUM        75038.574755  77066.139305   
                      PER DAY       

## Other Tips and Useful Information

### Counting the Number of Rows for Each Unique Value

Say we want to know how many salaried employees work in each borough in each fiscal year. From this description, we know we need to pass _Work Location Borough_ to the _index_ argument and _Fiscal Year_ to the _columns_ argument. How do we tell Pandas to count the number of employees in each unique combination of borough and year? Simply pass the string `'count'` as the aggfunc.

In [17]:
pd.pivot_table(data=dfPay[dfPay['Pay Basis'] == 'PER ANNUM'],
               index='Work Location Borough',
               columns='Fiscal Year',
               values='Base Salary',
               aggfunc='count')

Fiscal Year,2014,2015,2016,2017,2018,2019
Work Location Borough,,,,,,
BRONX,933.0,21182.0,21423.0,22431.0,22388.0,22699.0
BROOKLYN,NaN,42027.0,43208.0,41476.0,40716.0,41630.0
MANHATTAN,1452.0,197712.0,204959.0,204614.0,204775.0,215398.0
QUEENS,652.0,43211.0,45569.0,47918.0,47701.0,48372.0
STATEN ISLAND,107.0,5654.0,5786.0,5889.0,5861.0,6013.0


Let's take a look at the above code. The _index_ and _columns_ arguments should be self-explanatory. We passed the string `'count'` to the _aggfunc_ argument. The only confusing thing might be the _values_ argument. Why did I pass _Base Salary_ to that? Because Pandas needs to count something. Anything. In this code, we told Pandas to count the number of rows in the _Base Salary_ column for each unique value of 'Work Location Borough' and _Fiscal Year_. In theory, we could have counted any other column. To see this, consider the following code. It's identical to the code above, except for the _values_ argument.

In [18]:
pd.pivot_table(data=dfPay[dfPay['Pay Basis'] == 'PER ANNUM'],
               index='Work Location Borough',
               columns='Fiscal Year',
               values='Last Name',
               aggfunc='count')

Fiscal Year,2014,2015,2016,2017,2018,2019
Work Location Borough,,,,,,
BRONX,933.0,21182.0,21423.0,22431.0,22388.0,22699.0
BROOKLYN,NaN,42027.0,43208.0,41476.0,40716.0,41630.0
MANHATTAN,1452.0,197712.0,204959.0,204614.0,204775.0,215398.0
QUEENS,652.0,43211.0,45569.0,47918.0,47701.0,48372.0
STATEN ISLAND,107.0,5654.0,5786.0,5889.0,5861.0,6013.0


### The `pivot_table` function returns a DataFrame

The `pivot_table` function returns a DataFrame. That means you can save the pivot_table to a variable, change its contents, and use it in future computations. To see this, consider the following code:

In [19]:
myPivotTable = pd.pivot_table(data=dfPay[dfPay['Pay Basis'] == 'PER ANNUM'],
                              index='Work Location Borough',
                              columns='Fiscal Year',
                              values='Last Name',
                              aggfunc='count')
myPivotTable

Fiscal Year,2014,2015,2016,2017,2018,2019
Work Location Borough,,,,,,
BRONX,933.0,21182.0,21423.0,22431.0,22388.0,22699.0
BROOKLYN,NaN,42027.0,43208.0,41476.0,40716.0,41630.0
MANHATTAN,1452.0,197712.0,204959.0,204614.0,204775.0,215398.0
QUEENS,652.0,43211.0,45569.0,47918.0,47701.0,48372.0
STATEN ISLAND,107.0,5654.0,5786.0,5889.0,5861.0,6013.0


In [20]:
# Get the type of the variable myPivotTable
type(myPivotTable)

pandas.core.frame.DataFrame

In [21]:
# Change the column names of myPivotTable
myPivotTable.columns = ['FY2014', 'FY2015', 'FY2016', 'FY2017', 'FY2018', 'FY2019']
myPivotTable

,FY2014,FY2015,FY2016,FY2017,FY2018,FY2019
Work Location Borough,,,,,,
BRONX,933.0,21182.0,21423.0,22431.0,22388.0,22699.0
BROOKLYN,NaN,42027.0,43208.0,41476.0,40716.0,41630.0
MANHATTAN,1452.0,197712.0,204959.0,204614.0,204775.0,215398.0
QUEENS,652.0,43211.0,45569.0,47918.0,47701.0,48372.0
STATEN ISLAND,107.0,5654.0,5786.0,5889.0,5861.0,6013.0


In [22]:
# Add a projection for FY2020
myPivotTable['FY2020 (projected)'] = myPivotTable['FY2019'] * 1.05
myPivotTable

,FY2014,FY2015,FY2016,FY2017,FY2018,FY2019,FY2020 (projected)
Work Location Borough,,,,,,,
BRONX,933.0,21182.0,21423.0,22431.0,22388.0,22699.0,23833.95
BROOKLYN,NaN,42027.0,43208.0,41476.0,40716.0,41630.0,43711.50
MANHATTAN,1452.0,197712.0,204959.0,204614.0,204775.0,215398.0,226167.90
QUEENS,652.0,43211.0,45569.0,47918.0,47701.0,48372.0,50790.60
STATEN ISLAND,107.0,5654.0,5786.0,5889.0,5861.0,6013.0,6313.65
